In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from custom import weather

# matplotlib.rcParams['font.family'] = ['Heiti TC']

def rmax(maxrow: int=50):
    pd.set_option('display.max_rows', maxrow)
def cmax(maxcol: int=50):
    pd.set_option('display.max_columns', maxcol)

%load_ext autoreload
%autoreload 2

## Get all station codes (所有測站代碼）

In [2]:
# url = "https://e-service.cwb.gov.tw/wdps/obs/state.htm"
# df = pd.read_html(url)
# df = pd.DataFrame(df[0])
# df.columns = ['stn_code', 'stn_name', 'altitude', 'Longitude', 'Latitude', 'city', 'address', 'data_start_date', 'stn_end_date', 'comment', 'orig_stn_code', 'new_stn_code']
#df.to_csv('all_station_20201126.csv', index=False)

In [3]:
# datastart = '2015/1/1'
# dataend = '2020/11/26'

# df['data_start_date2'] = df.data_start_date.map(lambda x: max(pd.to_datetime(x), datetime.strptime(datastart, "%Y/%m/%d")))

# df['data_duration'] = 12 * (datetime.strptime(dataend, "%Y/%m/%d").year - df.data_start_date2.dt.year)+\
#                     (datetime.strptime(dataend, "%Y/%m/%d").month - df.data_start_date2.dt.month)

# df['data_duration2'] = df.data_duration.map(lambda x: f'{x//12}年{x%12}個月')
# df['data_period'] = df.data_start_date.map(lambda x: f'{x}-2020/11/26')

### load from saved dataframe

In [4]:
df = pd.read_csv('all_station_20201126.csv')
df

,stn_code,stn_name,altitude,Longitude,Latitude,city,address,data_start_date,stn_end_date,comment,orig_stn_code,new_stn_code,data_start_date2,data_duration,data_duration2,data_period
0,466850,五分山雷達站,756.0,121.781205,25.071182,新北市,瑞芳區靜安路四段1巷1號,1998/07/01,NaN,本站只有雷達觀測資料。,NaN,NaN,2015-01-01,70,5年10個月,1998/07/01-2020/11/26
1,466880,板橋,9.7,121.442017,24.997647,新北市,板橋區大觀路二段265巷62號,1972/03/01,NaN,原為探空站，自2002年開始進行氣象觀測。因受附近興建合宜住宅影響，日照觀測自民國104年起...,NaN,NaN,2015-01-01,70,5年10個月,1972/03/01-2020/11/26
2,466900,淡水,19.0,121.448906,25.164889,新北市,淡水區中正東路42巷6號,1942/01/01,NaN,NaN,NaN,NaN,2015-01-01,70,5年10個月,1942/01/01-2020/11/26
3,466910,鞍部,837.6,121.529731,25.182586,臺北市,北投區陽明山竹子湖路111號,1937/01/01,NaN,NaN,NaN,NaN,2015-01-01,70,5年10個月,1937/01/01-2020/11/26
4,466920,臺北,5.3,121.514853,25.037658,臺北市,中正區公園路64號,1896/01/01,NaN,2014至2015氣候年報之氣壓計海面上高度應為30.3m(2014/10/25搬遷，6.6...,NaN,NaN,2015-01-01,70,5年10個月,1896/01/01-2020/11/26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,C1Z110,三棧,680.0,121.605010,24.115570,花蓮縣,秀林鄉雙露山,2017/12/01,NaN,原(C1Z090)站因移位，於2017/12/1變更為(C1Z110)站。,C1Z090,NaN,2017-12-01,35,2年11個月,2017/12/01-2020/11/26
604,C1Z120,壽豐,62.0,121.508081,23.870853,花蓮縣,壽豐鄉壽豐村壽山路37號(壽豐鄉文康中心),2017/06/03,NaN,原(C1T890)站因移位，於2017/6/3變更為(C1Z120)站。,C1T890,NaN,2017-06-03,41,3年5個月,2017/06/03-2020/11/26
605,C1Z130,銅門,187.0,121.492842,23.965694,花蓮縣,秀林鄉銅門村69號(銅門國小),2017/06/03,NaN,原(C1T860)站因移位，於2017/6/3變更為(C1Z130)站。,C1T860,NaN,2017-06-03,41,3年5個月,2017/06/03-2020/11/26
606,C1Z140,荖溪,950.0,121.459640,23.890575,花蓮縣,秀林鄉,2018/03/23,NaN,NaN,NaN,NaN,2018-03-23,32,2年8個月,2018/03/23-2020/11/26


In [5]:
## station with data less than 1 year
new_stn = df[df.data_duration<12]
new_stn.shape

(22, 16)

## Show the location of weather station on map

In [6]:
## upgrade to the latest commit of brance and folium to fix unicode character display problem
# !pip install git+https://github.com/python-visualization/branca.git@master

In [7]:
# !pip install git+https://github.com/python-visualization/folium.git@master

In [8]:
# !pip install --force-reinstall branca folium

In [9]:
import branca

branca.__version__

'0.4.1+4.gac45f1e'

In [10]:
import folium

folium.__version__

'unknown'

In [12]:
### show all weather station. Not run 
# map_weather = map_with_marker(lat=df['Latitude'], lng=df['Longitude'], label1=df['stn_name'], 
#                               label2=df['city'], color=df['city'].astype('category').cat.codes, 
#                               cmap='tab10', location='Taiwan', zoom=8)
# map_weather

## Load cleaned weather data

In [11]:
df_w = pd.read_pickle('weather_all_cleaned.bz2')

In [12]:
# only keep weather stations without too many missing data (as filtered in the cleaned dataset)
code_list = df_w.index.get_level_values(0).unique().values

df2 = df[df.stn_code.isin(code_list)]

### plot again filtered station on map

In [15]:
# map_weather = weather.map_with_marker(lat=df2['Latitude'], lng=df2['Longitude'], label1=df2['stn_name'], 
#                               label2=df2['city'], color=df2['city'].astype('category').cat.codes, 
#                               cmap='tab10', location='Taiwan', zoom=8)
# map_weather

## Plot wind directions

In [16]:
# code = '466910'
# temp = df_w.loc[code]
# stn_name = df[df.iloc[:,0]==code].iloc[0,1]

# temp['Month'] = temp.index.month_name().astype('category')

# fig, axes = plt.subplots(figsize=(20,15), ncols=4, nrows=3, subplot_kw=dict(polar=True))
# fig.suptitle(stn_name, fontsize=24)
# for (i, m), ax in zip(enumerate(temp.Month.unique()), axes.flat):
#     wind = temp[temp.Month==m].WD
#     weather.plot_wind(wind, 10, ax=ax)
#     ax.set_title(m, pad=20)

# plt.tight_layout()

### plot and save wind directon plot for all stations

In [13]:
## filter out station with <1y data
codes_plot = list(set(df2.stn_code.unique()).difference(set(new_stn.stn_code.unique())))
len(codes_plot)

446

In [ ]:
for code in tqdm(codes_plot):

    stn_name = df[df.iloc[:,0]==code].iloc[0,1]
    print(f'plotting {stn_name}')
    
    temp = df_w.loc[code]
    temp['month'] = temp.index.month.astype('category')
    temp['year'] = temp.index.year.astype('category')

    fig, axes = plt.subplots(figsize=(12,15), ncols=3, nrows=4, subplot_kw=dict(polar=True))
#     fig.suptitle(stn_name, fontsize=18, y=0.98)
    bw=10
    y_max=0
    for m, ax in zip(np.r_[12,1:12], axes.flat):
        temp_m = temp[temp.month==m]
        wind = temp_m.WD
        windG = temp_m.WDGust

        ## Get the max wind gust speed and dir in full period (all years)
        idx = temp_m.WSGust.idxmax()
        G_max = temp_m.loc[idx, 'WSGust']
        G_dir = temp_m.loc[idx, 'WDGust']
    # #     counts, bins = np.histogram(temp_m.loc[idx.dropna()].WDGust, bins=(360//bw), range=[0,360])
    #     ## 當月最大風速 "最常" 出現在那個方向.
    #     G_dir = bins[np.argmax(counts)]

        radi_max = weather.plot_wind_gust(wind, windG, G_dir, G_max, bin_degree=10, ax=ax)
        y_max = radi_max if (radi_max>y_max) else y_max
        monthname = datetime.strptime(str(m), "%m").strftime("%B")
        ax.set_title(monthname, x=0.1)
    
    for ax in axes.flat:  ## set all subplot with the same maximum
        ax.set_ylim(0,y_max)
        
    plt.tight_layout()
    plt.savefig(f'resources/{code}.png')
    plt.close(fig)

## Plot map with wind stat image popup

In [57]:
from matplotlib import cm
import folium
import geocoder
import os
import base64


def map_with_windpopup(df, c_col, imagepath, cmap='Set1', location='Taiwan', zoom=9, jupyter=True):
    '''
    Create folium map with popup showing pre-generate images.
    Depending on whether the map need to be rendered in jupyter or in browser, 
    the way to embed image differs. In jupyter images need to be encoded in base64 
    string and save along the map html file. But for render in browser, the html
    link still works, and we don't need to embed images in html files but can be 
    loaded separately from img src="link".
    '''
    # create a color map
    ccode = df[c_col].astype('category').cat.codes
    cmap = cm.get_cmap(cmap, ccode.nunique())    # PiYG
    rgb = [matplotlib.colors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)]
    
    # create the location map
    g = geocoder.osm(location)
    latitude = g.latlng[0]
    longitude = g.latlng[1]
    
    # create the main map
    f = folium.Figure(width=800, height=900)
    map_ = folium.Map(location=[latitude, longitude], zoom_start=zoom).add_to(f)    
    
    # add marker to map
    for c,(i,row) in zip(ccode, df.iterrows()):
        hovertext = f'{row.stn_name}, {row.city}, Alt {row.altitude}m'
        Filename =os.path.join(imagepath, f'{row.stn_code}.png')
        html = f"<b>{row.stn_name}{row.stn_code}, 資料期間:{row.data_duration2}</b>"
        if os.path.isfile(Filename):
            if jupyter:
            ## embed the image as base64 string inside the html file
                encoded = base64.b64encode(open(Filename, 'rb').read())
                html+=f'<img src="data:image/png;base64,{encoded.decode("UTF-8")}" style="width:98%;height:98%;">'
                html=folium.IFrame(html, width=600, height=700)
#                 popup=folium.Popup(iframe, max_width='100%', autoPan=True, keepInView=True)
            else:
            # load the image in resources folder
                html += f'<img src="{Filename}" width="600" height="700">'
        else:
            print(f'file not exist:{Filename}')    
        popup=folium.Popup(html, max_width='100%', keepInView=True)
        
        folium.CircleMarker(
            [row.Latitude, row.Longitude],
            radius=5,
            popup=popup,
            tooltip=hovertext,
            color=rgb[c],
            fill=True,
            fill_color=rgb[c],
            fill_opacity=0.9,
            parse_html=False).add_to(map_)
#         print(i, row.stn_name)
    
    return map_

In [58]:
map_weather = map_with_windpopup(df2, c_col='city', imagepath='resources', 
                                 cmap='tab10', location='Taiwan', zoom=8, jupyter=False)
map_weather

file not exist:resources/467270.png
file not exist:resources/C0E880.png
file not exist:resources/C0FA00.png
file not exist:resources/C0G910.png
file not exist:resources/C0M860.png
file not exist:resources/C0R710.png
file not exist:resources/C0R730.png
file not exist:resources/C0R740.png
file not exist:resources/C0R750.png
file not exist:resources/C0R760.png
file not exist:resources/C0R770.png
file not exist:resources/C0R780.png
file not exist:resources/C0R790.png
file not exist:resources/C0R800.png
file not exist:resources/C0R810.png
file not exist:resources/C0R820.png
file not exist:resources/C0R830.png
file not exist:resources/C0R840.png
file not exist:resources/C0SA60.png
file not exist:resources/C0SA90.png
file not exist:resources/C0X320.png


In [59]:
map_weather.save('index.html')

In [17]:
"C0R710" in codes_plot

False